# Импорт библиотек

In [1]:
import numpy as np
from gzip import open # NB: overrides standard open()
import pickle as pkl
import pandas as pd
import warnings 
from glob import glob
import os
import math
import functools

### Отключим предупреждения Anaconda

In [2]:
warnings.simplefilter('ignore')

# Загрузим данные

In [3]:
Xdata_numpy = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/X.pkl.gz', 'rb'))
Ydata = pkl.load(open('C:/Users/admin/Desktop/PLS_continue_2020/y.pkl.gz', 'rb'))

# N-PLS1

## Выберем столбец у с которым будем работать

In [4]:
m=0

# Приближение к правильному алгоритму

In [104]:
from sklearn.model_selection import train_test_split

In [219]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdata_numpy["X"], Ydata.iloc[:,m], test_size=0.2857 
    )

In [220]:
X_train.shape[0]

25

In [260]:
x=X_train

In [261]:
y=y_train

In [262]:
y=np.array(y)

In [263]:
x=np.array(x)

In [264]:
yo=y

In [232]:
# a-numpy array;  b-pandas array
def error(a,b):
    s=0
    for i in range(0,len(a)):
        s+=((a[i]-b.iloc[i]))**2
    return s

In [156]:
def function_1(x,y,fn):
    Tt=[]
    for f in range(0,fn):
        z=np.zeros([x.shape[1],x.shape[2]])
        for i in range(0,x.shape[2]):
            for j in range(0,x.shape[1]):
                zsum=0
                for k in range(0,x.shape[0]):
                    zsum= zsum+ x[k,j,i]*y[k]
                z[j,i]=zsum
        Wk, S, WI = np.linalg.svd(z)
        w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
        w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
        t=[]
        for h in range(0,x.shape[0]):
            t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
        t=np.array(t).reshape(x.shape[0],1)
        Tt+=[t]
        T=np.array(Tt).reshape(x.shape[0],f+1)
        bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    
        for g in range(0,x.shape[0]):
            x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
        print(y[1])
        y=y-(np.dot(T,bf)) 
    return y

#### Ниже изначальная, но, пока ещё нужная для работы версия function_1

###### Массив mass собирает разложение обратно, и именно он важен для определения числа комронент

In [265]:
Tt=[]
mass=np.zeros([y.shape[0]])
for f in range(0,55):
    z=np.zeros([x.shape[1],x.shape[2]])
    for i in range(0,x.shape[2]):
        for j in range(0,x.shape[1]):
            zsum=0
            for k in range(0,x.shape[0]):
                zsum= zsum+ x[k,j,i]*y[k]
            z[j,i]=zsum
    Wk, S, WI = np.linalg.svd(z)
    w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
    w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
    
    t=[]
    for h in range(0,x.shape[0]):
        t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
    t=np.array(t).reshape(x.shape[0],1)
    Tt+=[t]
    T=np.array(Tt).reshape(x.shape[0],f+1)
    bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
    for g in range(0,x.shape[0]):
        x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
    y=y-(np.dot(T,bf))
    mass+=np.dot(T,bf)
    print(mass-yo)

[  1.05698837  -3.35637826  -9.56812821  16.66822433  13.52114069
  -9.85546677   6.15256353  -4.52494658 -31.52792228  27.27531748
 -18.18773578  -4.79661151 -25.9145256   23.95450816 -15.55845943
   8.79011704   2.33406467 -22.60683585 -24.16007717   6.42647951
  -6.41230467 -24.23476194  -9.14154252  -7.50016855   2.99742048]
[  5.22086251 -11.93694184   5.8317109   28.66690082  -1.12813694
 -10.21071171   3.76356705  -6.88324951 -22.67732784  19.83378063
  -9.33309756   0.9904668   -7.88917294  24.09739182 -15.36303406
   8.8242858    2.40360693 -22.36145592 -24.03978453   6.69915659
  -6.07119879 -23.95627345  -8.88606592  -7.15602564   3.35556582]
[  8.13347456  -5.93748843  10.2054777   37.90051327   0.67579533
  -3.89715832   4.07114437  -4.85816147 -18.1296568   19.87985918
  -9.30088957   0.99888031  -7.8270827   24.03737133 -15.24507851
   8.83508171   4.75559702 -14.84272979 -17.24725957  13.81962662
  -4.1172702  -15.9465175   -5.76770323  -1.94460713   6.22027537]
[  6.22

[ 1.09665876e-09 -1.86709954e-08  8.49474446e-09  1.05769686e-08
  6.40387299e-09  7.16255499e-09  1.22629018e-08  1.12292184e-08
  6.75404976e-09  6.07946760e-09  6.00704908e-09  2.20658407e-08
 -3.76006903e-06 -1.11697748e-05 -8.59880630e-06 -3.15150452e-06
 -2.84174175e-06 -5.36217080e-06 -1.30227224e-05  2.45111012e-05
  9.97923620e-06  8.05255114e-06  1.23530414e-05  1.53388345e-05
  2.05456803e-06]
[ 2.08189022e-12 -2.70290457e-11 -1.77723081e-09 -4.10746281e-09
 -4.15867163e-09 -3.10964055e-09 -5.10240028e-09 -6.44475051e-09
 -3.15666426e-09 -5.92228133e-09 -4.15207069e-09 -3.61477248e-09
 -1.09920961e-11  3.22337712e-11  5.07540676e-11  7.85078669e-11
  3.03970182e-11  4.67537120e-11  2.28084218e-12 -1.35713663e-11
 -8.89457397e-11  3.71684905e-11  8.82991458e-11 -9.95541427e-11
 -1.05622178e-11]
[ 3.47952778e-10 -1.12251541e-10  2.52398991e-10  1.88849469e-09
 -9.40246991e-10  2.45427145e-09  3.25790950e-09  4.25544755e-09
 -7.06705805e-10 -9.34150535e-11  1.01803721e-09  2.16

[-1.33286186e-02 -1.12081753e-02 -1.00519352e-02 -3.91804548e-03
 -7.82385397e-03 -2.53689371e-03 -7.36695943e-03 -6.19261738e-04
 -2.56478592e-04  2.69649234e-03  9.15677788e-04  1.68813432e-03
  2.80804089e-04 -4.08799705e-04 -6.95250839e-04  3.19106015e-05
  3.35121430e-04 -2.45960012e-03 -1.89944136e-04 -1.02298363e-04
 -2.96947318e-03 -1.33474122e-04  3.74375892e-04  7.79196161e-04
  1.67801912e-04]
[ 1.08813586e-02  7.35112014e-05  1.39724119e-03  2.14085984e-02
 -1.08684394e-02  7.93660724e-03  4.30551550e-03  8.20319960e-05
  3.34535279e-06  1.89830264e-04 -2.44414238e-04  1.87140058e-04
  2.07557963e-04  3.45145770e-04  1.40018113e-04 -3.56226857e-05
  1.27357633e-04  2.62448247e-04  5.11946254e-05  6.73990306e-05
 -2.96443380e-05  3.09659817e-05 -8.37438180e-05 -8.42688055e-05
  4.61335223e-05]
[-1.75509928e-01  1.35627722e+00 -3.76545969e-01  3.03072911e-01
 -5.10715643e-01  2.41373612e-01 -4.95324668e-01 -5.02974457e-03
 -2.37578680e-03 -3.57785394e-04 -2.58019839e-03 -1.45

# Попробую оформить всё в виде полноценного класса

In [ ]:
class N_PLS1():
    def error(a,b):
        s=0
        for i in range(0,len(a)):
            s+=((a[i]-b.iloc[i]))**2
        return s

    def fit(x,y,fn):
        Tt=[]
        mass=[]
        for f in range(0,fn):
            z=np.zeros([x.shape[1],x.shape[2]])
            for i in range(0,x.shape[2]):
                for j in range(0,x.shape[1]):
                    zsum=0
                    for k in range(0,x.shape[0]):
                        zsum= zsum+ x[k,j,i]*y[k]
                    z[j,i]=zsum
            Wk, S, WI = np.linalg.svd(z)
            w_k=np.array(Wk[0,:]).reshape(x.shape[1],1)
            w_i=np.array(WI[0,:]).reshape(x.shape[2],1)
            t=[]
            for h in range(0,x.shape[0]):
                t+=[np.dot(np.dot(w_k.transpose(),x[h,:,:]),w_i)]
            t=np.array(t).reshape(x.shape[0],1)
            Tt+=[t]
            T=np.array(Tt).reshape(x.shape[0],f+1)
            bf=np.dot(np.dot(np.linalg.inv(np.dot(T.transpose(),T)),T.transpose()),y)
            for g in range(0,x.shape[0]):
                x[g,:,:]=x[g,:,:]-(t[g]*w_i.transpose()*w_k)
            y=y-(np.dot(T,bf))
            
        return y

In [179]:
y

array([34.19308759, 35.72443085, 46.32211944, 35.69579348, 47.44184036,
       33.34204937, 42.44462856, 39.9018207 , 44.39973652, 54.64066407,
       53.71596802, 44.04933373, 36.11255278, 34.24246421, 36.22622795,
       26.1974747 , 41.0398005 , 31.41066637, 34.07047331, 42.14770231,
       45.44679116, 21.41118209, 44.54683093, 35.35460545, 36.47993911])

In [54]:
print(y.shape)

()


In [157]:
function_1(x,y,5)

-5.4763977852194214e-14
-1.420663662855037e-12
-1.4580267029464817e-12
-1.2938120676022591e-12
-1.1881446182327078e-12


array([ 1.68894612e-12,  2.00329419e-13,  1.32327231e-12,  3.07245841e-14,
        2.71218908e-12, -4.54736445e-13,  1.42743948e-12,  3.67873053e-12,
       -2.17012646e-12,  2.51672045e-12, -2.75503170e-12, -6.19494150e-12,
       -4.84098214e-12, -8.87287420e-13,  6.49672160e-12,  2.28468952e-12,
        1.99518072e-12,  3.14275609e-12, -1.87897603e-12,  8.34491128e-13,
       -2.95291505e-12,  1.31643761e-12,  3.38456727e-12,  3.07099108e-12,
       -8.91553773e-12])